In [7]:
PV = PolynomialRing(QQ, ['v%d'%i for i in range(20)])
Ph = PolynomialRing(PV, 'h')
Pd = PolynomialRing(Ph, 'd')
d = Pd.gen()
h = Pd(Ph.gen())
zero = Pd.zero()
V = list(map(Pd, PV.gens()))

def shifted_legendre(n):
    x = var('x')
    return Pd((maxima.legendre_p(n,2*d/h-1).sage()*h**n).expand())

def filter_high_degree(expr, n, depth=2):
    if depth == 0:
        return expr
    s = parent(expr).zero()
    v = d if depth == 2 else h
    for i, coef in enumerate(expr):
        if i > n:
            break
        s += filter_high_degree(coef, n-i, depth-1)*v**i
    return s

In [8]:
def calculateEta(hmax, n):

    numIt = (2*n)//3+1
    dV = sum(shifted_legendre(i)*V[i] for i in range(1, n))

    def calcC(Q, R0):
        C = [[integrate(Pd(Q/2), d)]]
        R0 = Pd(R0)
        for q in range(numIt):
            if q > 0:
                C.append([zero])
            for m in range(0, hmax//2+1):
                if q == 0:
                    R = R0 if m == 0 else zero
                else:
                    R = filter_high_degree(dV*C[q-1][m], hmax-2*m-1)
                
                C[q].append(
                    integrate(d**m * (
                        R - diff(C[q][m], d, d)
                    )*Pd(1/2), d)
                       .quo_rem(d**(m+1))[0]
                )
        return C

    Cu = calcC(dV, zero)
    Cv = calcC(zero, dV)

    eta_u = [zero for _ in range(hmax//2+2)]
    eta_up = eta_u[:]
    eta_v = eta_u[:]
    eta_vp = eta_u[:]

    eta_u[0] = Pd.one()
    eta_up[0] = zero
    eta_v[1] = d
    eta_vp[0] = Pd.one()

    for i in range(numIt):
        for k in range(hmax//2+1):
            eta_u[k+1] += filter_high_degree(Cu[i][k] * d**(2*k+1), hmax)
            eta_v[k+1] += filter_high_degree(Cv[i][k] * d**(2*k+1), hmax)
        eta_up[0] += Cu[i][0]
        eta_vp[0] += Cv[i][0]
        for k in range(hmax//2):
            eta_up[k+1] += filter_high_degree((diff(Cu[i][k], d) + d*Cu[i][k+1]) * d**(2*k+1), hmax)
            eta_vp[k+1] += filter_high_degree((diff(Cv[i][k], d) + d*Cv[i][k+1]) * d**(2*k+1), hmax)

    return [eta_u, eta_v, eta_up, eta_vp]

In [9]:
dhmax = 16
dn = 14
hhmax = 18
hn = 16

de = calculateEta(dhmax, dn)
print("calculated eta delta")
he = calculateEta(hhmax, hn)
print("calculated eta h")

calculated eta delta
calculated eta h


In [18]:
header = "#include \"./matslise_formulas.h\"\n\n"
header += "template<typename Scalar>\n"
header += "void calculate_tcoeff_matrix(\n"
header += "        const Scalar &h,\n"
header += "        const std::array<Scalar, MATSLISE_N> &vs,\n"
header += "        Eigen::Array<Eigen::Matrix<Scalar, 2, 2, Eigen::DontAlign>, MATSLISE_ETA_delta, MATSLISE_HMAX_delta, Eigen::DontAlign> &tDelta,\n"
header += "        Eigen::Matrix<Scalar, 2, 2, Eigen::DontAlign> *tH) {\n"

for i in range(1, max(hn, dn)):
    header += "    const Scalar &v%d = vs[%d];\n"%(i, i)

body = ""

def expr_to_C(expr, depth=2):
    if depth == 0:
        return str(float(expr.n()))
        return '(D('+str(expr.numerator())+'L)/D('+str(expr.denominator())+'L))'
    if depth == 1:
        vs = PV.gens()
        r = []
        for k, v in expr.dict().items():
            r.append(expr_to_C(v, depth-1) + "".join(
                ('*' + str(gen))*m for gen, m in zip(vs, k) if m > 0
            ))
        if len(r) == 0:
            return '0'
        return '+'.join(r)
    
    gen = 'd' if depth == 3 else 'h'
    r = list(expr)
    if len(r) == 0:
        return '0'
    elif len(r) == 1:
        return expr_to_C(r[0], depth-1)
    else:
        l = ''
        for i, v in enumerate(r):
            if v == 0:
                if i < len(r) - 1:
                    l += '(h*'
                else:
                    l += '(0'
            else:
                l += '(('+expr_to_C(v, depth-1)+')'
                if i < len(r)-1:
                    l += ' + h*'
        return l + ')'*len(r)

for j in range(1+hhmax//2):
    if j <= dhmax/2:
        exprs = [filter_high_degree(de[k][j], dhmax) for k in range(4)]
        for i in range(dhmax+1):
            body += "    tDelta(%d, %d)"%(j, i)
            body += " << " + ", ".join([expr_to_C(e[i]) for e in exprs]) + ";\n"
    h_exprs = [filter_high_degree(he[k][j], hhmax) for k in range(4)]
    body += "    tH[%d]"%(j)
    body += " << " + ", ".join([expr_to_C(e(h), 3) for e in h_exprs]) + ";\n"
        

In [19]:
from collections import defaultdict
import re
variable = "([a-z][a-zA-Z0-9_]*)"
pattern = re.compile("\\b"+variable+"\\s*\\*\\s*"+variable+"\\b")

head = []
replaced_body = body
replace_header = ""
while True:
    var_freqs = defaultdict(int)
    for v in pattern.findall(replaced_body):
        var_freqs[v] += 1
    if len(var_freqs) == 0:
        break
    m = max(var_freqs.keys(), key=lambda k: var_freqs[k])
    count = var_freqs[m]
    print(m)
    if count < 2:
        break
    v1, v2 = m
    name = v1+"_"+v2
    replaced_body = re.sub("\\b"+v1+"\\s*\\*\\s*"+v2+"\\b", name, replaced_body)
    replace_header += '    Scalar ' + name+ ' = '+v1+' * '+v2+';\n'

('v1', 'v1')
('v2', 'v2')
('v1', 'v2')
('v1', 'v3')
('v2', 'v3')
('v3', 'v3')
('v1', 'v4')
('v1', 'v2_v2')
('v2', 'v4')
('v1_v1', 'v1_v1')
('v5', 'v6')
('v4', 'v7')
('v3', 'v8')
('v2', 'v9')
('v1', 'v10')
('v6', 'v6')
('v5', 'v7')
('v3', 'v9')
('v1', 'v11')
('v4', 'v8')
('v2', 'v10')
('v1_v1', 'v1')
('v5', 'v5')
('v3', 'v4')
('v3', 'v7')
('v1', 'v9')
('v4', 'v6')
('v2', 'v8')
('v4', 'v4')
('v4', 'v5')
('v3', 'v6')
('v2', 'v7')
('v1', 'v8')
('v3_v3', 'v3')
('v1_v4', 'v4')
('v2_v2', 'v5')
('v1_v1', 'v7')
('v2_v3', 'v4')
('v1_v3', 'v5')
('v1_v2', 'v6')
('v3_v3', 'v4')
('v2_v4', 'v4')
('v2_v3', 'v5')
('v1_v4', 'v5')
('v1_v1', 'v8')
('v2_v2', 'v6')
('v1_v3', 'v6')
('v1_v2', 'v7')
('v3', 'v5')
('v1', 'v7')
('v2_v2', 'v4')
('v1_v3', 'v4')
('v2', 'v6')
('v2_v3', 'v3')
('v1_v1', 'v6')
('v1_v2', 'v5')
('v2_v2', 'v2_v2')
('v1_v1', 'v3_v3')
('v2', 'v5')
('v1', 'v6')
('v2_v2', 'v3')
('v1_v3', 'v3')
('v1_v1', 'v5')
('v1_v2_v2', 'v3')
('v1_v1', 'v2_v4')
('v1_v1_v1', 'v5')
('v1_v2', 'v4')
('v1_v2_v2',

In [28]:
footer = "}\n\n"
footer += "#define INSTANTIATE_MORE(Scalar) \\\n"
footer += "    template void calculate_tcoeff_matrix<Scalar>( \\\n"
footer += "        const Scalar &, \\\n"
footer += "        const std::array<Scalar, MATSLISE_N> &, \\\n"
footer += "        Eigen::Array<Eigen::Matrix<Scalar, 2, 2, Eigen::DontAlign>, MATSLISE_ETA_delta, MATSLISE_HMAX_delta, Eigen::DontAlign> &, \\\n"
footer += "        Eigen::Matrix<Scalar, 2, 2, Eigen::DontAlign> *tH \\\n"
footer += ");\n\n"
footer += '#include "instantiate.h"\n'

with open('matslise_formulas.cpp', 'w') as f:
    f.write(header + replace_header + replaced_body+footer)